In [1]:
import pandas as pd 
import re  
import praw

In [ ]:
# Connect to Reddit via API

reddit = praw.Reddit( 

    client_id = "your_client_id",
    client_secret = "your_key",
    user_agent = "your_username"
)

In [ ]:
# Call this function to remove URLs
def clean(text): 
    text = re.sub("http\S+|www\S+|https\S+", "", text)
    return text

In [ ]:
# Fetch top 10 comments from a subreddit's hottest posts 
def collect_reddit_data(subreddit=str, post_num=int):
    data = [] 
    
    for submission in reddit.subreddit(subreddit).hot(limit=post_num):

        if 'string_to_filter_out' in submission.title.lower(): # Filter out unwanted threads
            continue
    
        post_title = clean(submission.title) 

        submission.comments.replace_more(limit=0) # Fetch data from the entire comment forest 
        top_comments = sorted(submission.comments.list(), key=lambda x: x.score, reverse=True)[:10]

        for comment in top_comments: # Remove deleted comments and bots 
            if "bot" in comment.body or comment.body in ["[deleted]", "[removed]"]: 
                continue
            comment_text = clean(comment.body) 
            upvotes = comment.score 
            
            
            data.append({ # Store the data
                'post_title': post_title,
                'comment_text': comment_text,
                'upvotes': upvotes
            })
    
    return pd.DataFrame(data) # Return the results in a data frame

df = collect_reddit_data('sample_subreddit', 1000) # Call the function with the subreddit name and the limit number of post 

df.to_csv('/your_path/your_dataset.csv', index=True) # Optional: store the results in a CSV file
print(df.head()) # Inspect the dataset 

